<a href="https://colab.research.google.com/github/Chiwidude/ETL-EstimadorRiesgo/blob/dscience/ETLv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparación de Entorno**

In [ ]:
#ejecutar sino se tienen instaladas estas librerías
!pip install -U scikit-learn
!pip install scikit-learn-intelex
!pip install -U imbalanced-learn
!pip install xgboost
!pip install seaborn
!pip install Pyspatialml
!pip install scikit-learn-intelex
!pip install scikit-optimize

In [ ]:
from pyspatialml import Raster
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Conjuntos de datos variables independientes

In [ ]:
#Datos variables independendientes

files = ["drive/MyDrive/Tesis/raster_curvatura/curvatura_r.tif","drive/MyDrive/Tesis/raster_elevacion/elevacion_r.tif","drive/MyDrive/Tesis/raster_pendiente/pendiente_r.tif","drive/MyDrive/Tesis/raster_perfil_curvatura/perfilc_r.tif",
         "drive/MyDrive/Tesis/raster_lluvia/lluvia_r.tif","drive/MyDrive/Tesis/raster_cobertura/cobertura_r.tif"]
rasters =Raster(files)

In [ ]:
#Datos deslizamientos
raster_desli = rasterio.open('drive/MyDrive/Tesis/deslizamientos/raster/desli_lim.tif')

In [ ]:
rasters.bounds

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

rasters.cobertura_r.plot(ax=ax)
rasterio.plot.show(raster_desli, ax=ax)
plt.show()

In [ ]:
#Limpieza de datos NA
extracted_df = rasters.extract_raster(raster_desli)

extracted_df.dropna(inplace=True)
extracted_df

# Pipeline Ciencia de Datos (ML)

In [ ]:
#Data Spliting inicial
#Sobremuestreo de datos
from sklearnex import patch_sklearn 
patch_sklearn()
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score

## Random Forest

In [ ]:
#variables independientes
X = extracted_df.drop(columns=['value','geometry']).values

#variable dependiente (deslizamientos)
Y = extracted_df['value'].values

smote = SMOTE(random_state=30, n_jobs=-1)

X_smote, Y_smote = smote.fit_resample(X,Y)

x_train, x_test, y_train, y_test = train_test_split(X_smote,Y_smote,test_size=0.2, random_state=30)

In [ ]:
#Búsqueda mejor ajuste hiperparámetros
base_classifier = RandomForestClassifier()

n_estimators = [100,250,500,750,1000,1200]

max_depth = [50,100, 250, 350, 500]

min_samples_split = [5, 10, 15, 20, 30]

min_samples_leaf = [2,4,6,10]

max_leaf_nodes= [20, 50, 100,150,200]



params_grid = {'n_estimators':n_estimators,
               'max_depth':max_depth,
               'min_samples_split':min_samples_split,
               'min_samples_leaf':min_samples_leaf,
              'max_leaf_nodes':max_leaf_nodes}

rf_sampled = GridSearchCV(estimator=base_classifier,
                                param_grid=params_grid,
                                cv=5, verbose=2, n_jobs = -1,
                                return_train_score=True,
                          scoring=['f1','precision','recall'],
                          refit='recall'                 
                                )

rf_sampled.fit(x_train, y_train)

In [ ]:
params = {'n_estimators':500,
               'max_depth':250,
               'min_samples_split':5,
               'min_samples_leaf':4,
              'max_leaf_nodes':50}

params['n_jobs'] = -1

params

### Entrenamiento Hiperparámetros ajustados


In [ ]:
classifier = RandomForestClassifier(**params)

rf = Pipeline(
    [('scaling', StandardScaler()),
        ('classifier', classifier)])

rf.fit(x_train, y_train)

In [ ]:
#Resultados predicción
result_rf = rasters.predict(estimator=rf,file_path="drive/MyDrive/Tesis/prediction.tif", progress=True)

result_probs = rasters.predict_proba(estimator=rf, progress=True)

In [ ]:
#Plot mapa predicciones
fig, ax = plt.subplots(figsize=(9, 9))

result_rf.iloc[0].cmap = "Dark2"
result_rf.iloc[0].categorical = True
result_rf.iloc[0].plot(ax=ax, legend=True, categorical=True)
plt.show()

In [ ]:
y_pred_ajustado = rf.predict(x_test)

y_ajustado_proba = rf.predict_proba(x_test)[::,1]

In [ ]:
result_rf = rasters.predict(estimator=rf)

result_probs = rasters.predict_proba(estimator=rf)

### Entrenamiento Hiperparámetros default


In [ ]:
b_classifier = RandomForestClassifier(n_jobs=-1)

brf = Pipeline([('scaling', StandardScaler()),
        ('classifier', b_classifier)])

brf.fit(x_train, y_train)

In [ ]:
#Resultados predicción
result_rf = rasters.predict(estimator=brf, progress=True)

result_probs = rasters.predict_proba(estimator=brf, progress=True)

In [ ]:
#Plot mapa predicciones
fig, ax = plt.subplots(figsize=(9, 9))

result_rf.iloc[0].cmap = "Dark2"
result_rf.iloc[0].categorical = True
result_rf.iloc[0].plot(ax=ax, legend=True)
plt.show()

In [ ]:
y_pred_base = brf.predict(x_test)

y_pbase_proba = brf.predict_proba(x_test)[::,1]

### Métricas

In [ ]:
def plot_result(x_label, y_label, train_data, val_data, ax):
        '''Función para graficar un gráfico de barras agrupado mostrando datos de entrenamiento y validación
          resultantes del modelo en cada iteración aplicando validación cruzada de k -iteraciones.
         Parametros
         ----------
         x_label: str, 
            Nombre del algoritmo utilizado. ej:Random Forest
          
         y_label: str, 
            Nombre de la métrica a visualizar. ej: precisión
         plot_title: str, 
            Título del gráfico. ej: 'Precisión del modelo'
         
         train_result: list, array
            Resultados de la métrica en fase de entrenamiento del modelo en la validación cruzada.
        
         val_result: list, array
            Resultados de la métrica en fase de validación del modelo en la validación cruzada.
         Returns
         -------
         Gráfico de barras agrupado mostrando el resultado de la métrica deseada en cada iteración.
        '''
        
        labels = ["1st Fold", "2nd Fold", "3rd Fold", "4th Fold", "5th Fold"]
        X_axis = np.arange(len(labels))
        ax.bar(X_axis-0.2, train_data, 0.4, color='blue', label='Training')
        for x,y in zip(X_axis-0.2, train_data):
            label = "{:.2f}".format(y)
            ax.annotate(label,
                       (x,y),
                 textcoords="offset points",
                 xytext=(0,2),
                 ha='center',
                       fontsize=10)
        ax.bar(X_axis+0.2, val_data, 0.4, color='red', label='Validation')
        for x,y in zip(X_axis+0.2, val_data):
            label = "{:.3f}".format(y)
            ax.annotate(label,
                       (x,y),
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center',
                       fontsize=10)
        ax.set_xticks(X_axis, labels)
        ax.set_xlabel(x_label, fontsize=10)
        ax.set_ylabel(y_label, fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score

#### Matríz de Confusión + Métricas

In [ ]:
#Matriz de confusión parámetros default
matrix = confusion_matrix(y_test, y_pred_base)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:,np.newaxis]

plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Oranges, linewidths=0.2)


class_names = ['no deslizamiento','deslizamiento']
tick_marks = np.arange(len(class_names)) + 0.4
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=0)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Clase predecida')
plt.ylabel('Clase real')
plt.title('Matriz de Confusión Random Forest (param default)')
plt.show()

In [ ]:
#Resultado métricas de clasificación modelo parámetros pred.
print(classification_report(y_test, y_pred_base,digits=5))

In [ ]:
#Matriz de confusión parámetros ajustados
matrix = confusion_matrix(y_test, y_pred_ajustado)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:,np.newaxis]

plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Blues, linewidths=0.2)

# Add labels to the plot
class_names = ['no deslizamiento','deslizamiento']
tick_marks = np.arange(len(class_names)) + 0.4
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=0)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Clase predecida')
plt.ylabel('Clase real')
plt.title('Matriz de Confusión Random Forest (param ajustados)')
plt.show()

In [ ]:
#Resultado métricas de clasificación modelo parámetros ajustados
print(classification_report(y_test, y_pred_ajustado, digits=5))

#### AUC - ROC

In [ ]:
#Comparativa gráficos AUC-ROC RF

fpr, tpr, _ = roc_curve(y_test, y_pbase_proba)

auc = roc_auc_score(y_test, y_pbase_proba)

#curva ROC parámetros pred.

fig, axes = plt.subplots(nrows=1, ncols=2, figsize= (16,6))

axes[0].plot(fpr,tpr,label="AUC="+str(auc))
axes[0].set_title('default params')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_xlabel('False Positive Rate')
axes[0].legend(loc=4)

fpr, tpr, _ = roc_curve(y_test, y_ajustado_proba)

auc2 = roc_auc_score(y_test, y_ajustado_proba)

#curva ROC parámetros ajustados

axes[1].plot(fpr,tpr,label="AUC="+str(auc2))
axes[1].set_title('params ajustados')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_xlabel('False Positive Rate')
axes[1].legend(loc=4)
plt.show()

#### Validación Cruzada k - iteraciones

In [ ]:
#Validación modelo parámetros pred.
base_scores = cross_validate(
    estimator=brf,
    X=x_test,
    y=y_test,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    cv=5,
    return_train_score=True
)

for i, item in base_scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Validación modelo parámetros ajustados
scores = cross_validate(
    estimator=rf,
    X=x_test,
    y=y_test,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    cv=5,
    return_train_score=True
)

for i, item in scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Plot comparativa validación cruzada k - iteraciones RF
figure, axes = plt.subplots(nrows=4, ncols=2, figsize=(12,12))

metrics = ['accuracy', 'precision','recall','f1']

figure.suptitle('5-fold Validation', fontsize=24, fontweight='bold')

for index, metric in enumerate(metrics):
    pivot = 0
    plot_result('Random Forest (default params) test mean:'+str(round(base_scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',base_scores[f'train_{metric}'],base_scores[f'test_{metric}'], axes[index,pivot])
    pivot+=1
    plot_result(f'Random Forest (params ajustados)  test mean:'+str(round(scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',scores[f'train_{metric}'],scores[f'test_{metric}'], axes[index,pivot])
figure.tight_layout()
plt.show()

## Support Vector Machine

In [ ]:
#Data Spliting inicial
#Sobremuestreo de datos
from sklearnex import patch_sklearn 
patch_sklearn()
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from sklearn import svm

#Variables independientes
X = extracted_df.drop(columns=['value','geometry']).values
#Variable dependiente (deslizamientos)
Y = extracted_df['value'].values

smote = SMOTE(random_state=30, n_jobs=-1)

X_smote, Y_smote = smote.fit_resample(X,Y)

x_train, x_test, y_train, y_test = train_test_split(X_smote,Y_smote,test_size=0.2, random_state=30)

In [ ]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(x_train)

In [ ]:
#Búsqueda hiperparámetros
param_grid = {'C': [1,2,2.5,3,4,5],
              'gamma': [1,1.25,1.50,1.75],
              'kernel': ['rbf','linear','sigmoid'],
             'cache_size':[1000]}
base_svm = svm.SVC()

grid = GridSearchCV(base_svm,param_grid, n_jobs = -1,                                
                          scoring=['f1','precision','recall'], cv=5,
                          refit='recall', verbose=2)

grid.fit(Xtrain, y_train)

In [ ]:
params_svm = grid.best_params_
params_svm

### Entrenamiento Hiperparámetros ajustados

In [ ]:
svmc = svm.SVC(**params_svm, probability=True)

svc = Pipeline(
    [('scaling', StandardScaler()),
        ('classifier', svmc)])

svc.fit(x_train, y_train)

In [ ]:
#Resultado Predicción
result = rasters.predict(estimator=svc, progress=True)

result_probs = rasters.predict_proba(estimator=svc, progress=True)

In [ ]:
#Plot mapa con predicciones
fig, ax = plt.subplots(figsize=(9, 9))

result.iloc[0].cmap = "Dark2"
result.iloc[0].categorical = True
result.pred_raw_0.plot(ax=ax, legend=True)
plt.show()

In [ ]:
y_pred_svm = svc.predict(x_test)

y_svm_proba = svc.predict_proba(x_test)[::,1]

### Entrenamiento parámetros default

In [ ]:
base_svm = svm.SVC(probability=True)
base_svm = Pipeline(
    [('scaling', StandardScaler()),
        ('classifier', base_svm)])


base_svm.fit(x_train, y_train)

In [ ]:
#Resultados predicción
result = rasters.predict(estimator=base_svm, progress=True)

result_probs = rasters.predict_proba(estimator=base_svm, progress=True)

In [ ]:
#Plot mapa predicciones
fig, ax = plt.subplots(figsize=(9, 9))

result.iloc[0].cmap = "Dark2"
result.iloc[0].categorical = True
result.pred_raw_0.plot(ax=ax, legend=True)
plt.show()

In [ ]:
y_pred_svmb = base_svm.predict(x_test)

y_svmb_proba = base_svm.predict_proba(x_test)[::,1]

### Métricas

#### Matríz de Confusión + métricas

In [ ]:
#Resultado métricas de validación modelo params. pred.
print(classification_report(y_test, y_pred_svm,digits=5))

In [ ]:
#Resultado métricas de validación modelo params. ajustados
print(classification_report(y_test, y_pred_svmb,digits=5))

In [ ]:
#Matriz confusión modelo params. predeterminados
matrix = confusion_matrix(y_test, y_pred_svmb)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:,np.newaxis]

plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Oranges, linewidths=0.2)

class_names = ['no deslizamiento','deslizamiento']
tick_marks = np.arange(len(class_names)) + 0.4
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=0)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Clase predecida')
plt.ylabel('Clase real')
plt.title('Matriz de Confusión SVM (param default)')
plt.show()

In [ ]:
#Matriz confusión modelo params. predeterminados
matrix = confusion_matrix(y_test, y_pred_svm)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:,np.newaxis]

plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Blues, linewidths=0.2)


class_names = ['no deslizamiento','deslizamiento']
tick_marks = np.arange(len(class_names)) + 0.4
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=0)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Clase predecida')
plt.ylabel('Clase real')
plt.title('Matriz de Confusión SVM (param ajustados)')
plt.show()

#### AUC - ROC

In [ ]:
#Gráfico comparativa AUC - ROC
fpr, tpr, _ = roc_curve(y_test, y_svmb_proba)

auc = roc_auc_score(y_test, y_svmb_proba)

#curva ROC modelo parámetros default

fig, axes = plt.subplots(nrows=1, ncols=2, figsize= (16,6))

axes[0].plot(fpr,tpr,label="AUC="+str(auc))
axes[0].set_title('default params')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_xlabel('False Positive Rate')
axes[0].legend(loc=4)

fpr, tpr, _ = roc_curve(y_test, y_svm_proba)

auc2 = roc_auc_score(y_test, y_svm_proba)

#curva ROC modelo parámetros ajustados

axes[1].plot(fpr,tpr,label="AUC="+str(auc2))
axes[1].set_title('params ajustados')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_xlabel('False Positive Rate')
axes[1].legend(loc=4)
plt.show()

#### Validación cruzada k-iteraciones

In [ ]:
#Validación modelo parámetros pred.
base_scores = cross_validate(
    estimator=base_svm,
    X=x_test,
    y=y_test,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    cv=5,
    return_train_score=True
)

for i, item in base_scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Validación modelo parámetros ajustados
scores = cross_validate(
    estimator=svc,
    X=x_test,
    y=y_test,
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    cv=5,
    return_train_score=True
)

for i, item in scores.items():
    print(f'{i} media: {item.mean()}')

In [ ]:
#Plot gráfico comparativa validación cruzada k-iteraciones
figure, axes = plt.subplots(nrows=4, ncols=2, figsize=(12,12))

metrics = ['accuracy', 'precision','recall','f1']

figure.suptitle('5-fold Validation', fontsize=24, fontweight='bold')

for index, metric in enumerate(metrics):
    pivot = 0
    plot_result('SVM (default params) test mean:'+str(round(base_scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',base_scores[f'train_{metric}'],base_scores[f'test_{metric}'], axes[index,pivot])
    pivot+=1
    plot_result(f'SVM (params ajustados)  test mean:'+str(round(scores[f'test_{metric}'].mean(),5)),f'{metric.capitalize()}',scores[f'train_{metric}'],scores[f'test_{metric}'], axes[index,pivot])
figure.tight_layout()
plt.show()